In [1]:
'''
1.定义神经网络学习率以及权重
2.遍历数据集的输入
3.神经网络对数据进行处理
4.计算对应的损失
5.反向传播神经网络梯度的参数
6.更新神经网络的权重
'''
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        #Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
        #默认stride=(1,1),每次行的长度加上1，列的长度加上1
        #1为输入的通道，6为输出的通道，3为卷积核的尺寸
        #in_channel为1，outchannel为6，对应的卷积结果为3
        #这里的in_channel就是之前的outchannel，如果是第一层
        #的话就是开始的out_channel
        
        #这里的channel可以理解为图形的样本数量，
        self.conv2 = nn.Conv2d(6, 16, 3)
        #第二个卷积核对应的第一个参数为6，为输入的通道
        #第二个参数16为输出的通道，第三个参数3为卷积核的尺寸
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        #输入对应的第一层为576，由576转为120
        self.fc2 = nn.Linear(120, 84)
        #输入对应的第二层为由120转为84
        self.fc3 = nn.Linear(84, 10)
        #输入对应的第三层为由84转为10
        
        #!!!注意：这上面的操作只是定义了下面需要使用的参数，
        #并没有进行具体的神经网络的操作

    #之前操作的时候，这里输入的为([1,1,32,32])
    def forward(self, x):
        # Max pooling over a (2, 2) window
        print('forward = ***')
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        #self.conv1(x)为(1,6,3),所以需要6个卷积核进行操作，
        # If the size is a square you can only specify a single number
        #原始的数据为1*32*32的对应矩阵，使用3*3卷积核进行放缩
        #32-3+1=28，28/2=14，6*14*14
        #详情可以看这篇https://www.jianshu.com/p/45a26d278473
        print(x.size())
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        #self.conv1(x)为(6,16,3),操作完之后x
        #使用14-3+1=12，再经过一个max_pool2d之后为12/2=6，
        #由于使用了16个对应的卷积核，所以结果为6*6*16
        print(x.size())
        x = x.view(-1, self.num_flat_features(x))
        #将输出的6*6*16铺平之后结果为576，这里self.num_flat_features(x)
        #输出的对应结果为576，是所有值相乘
        print(x.size())
        #xprint('change x = ***')
        #如果是torch.view(-1),则原向量会变成一维的结构
        #t = torch.tensor([-0.3,-0.6,0.7,0.4,2.3,0.15])
        #result = t.view(3,2)
        #此时result变成([[-0.3 -0.6],
        #               [0.7,0.4],
        #               [2.3,0.15]])
        ##view函数将张量x变形成一维的向量形式，
        #总特征数并不改变，为接下来的全连接作准备
        x = F.relu(self.fc1(x))
        #变换结果为[120,576]*[576,1]=[120]
        print(x.size())
        x = F.relu(self.fc2(x))
        #变换结果为[84,120],[84]
        print(x.size())
        x = self.fc3(x)
        #变换结果为[10,84],[84]
        print(x.size())
        return x

    def num_flat_features(self, x):
        print('num_flat_features')
        size = x.size()[1:]  # all dimensions except the batch dimension
        print('size = ***')
        print(size)
        num_features = 1
        for s in size:
            num_features *= s
        print('num_features = ***')
        print(num_features)
        return num_features


net = Net()
print(net)
#这里将神经网络的结构打印出来，
#从而得到相应的每一层的结构

#stride中x表示卷积核的水平滑动步长
#y表示卷积核的垂直滑动步长

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [2]:
#定义了对应的forward前向传播函数，以及backward后向传播函数
params = list(net.parameters())
#学习的参数由net.parameters()函数进行返回
print('params = ***')
for  i  in  range(0,len(params)):
    print(params[i].size())
    print('###')
print('***')
#这里面的params没看明白里面包含的参数内容
#因为第一层为Conv2d(1,6,kernel_size=(3,3)),所以
#第一层中共有6个对应的3*3矩阵
#第二层为Conv2d(6,16,kernel_size=(3,3)),所以
#第二层中共有6*16=96个对应的3*3矩阵
#然后由于由in_features中的576转到out_features中对应的120
#所以这里面需要乘上一个576*120的对应的矩阵
#然后由于由in_features中的120转到out_features中对应的84
#所以这里面需要乘上一个120*84的对应的矩阵
#然后由于由in_features中的84转到out_features中对应的10
#所以这里面需要乘上一个84*10的对应的矩阵


params = ***
torch.Size([6, 1, 3, 3])
###
torch.Size([6])
###
torch.Size([16, 6, 3, 3])
###
torch.Size([16])
###
torch.Size([120, 576])
###
torch.Size([120])
###
torch.Size([84, 120])
###
torch.Size([84])
###
torch.Size([10, 84])
###
torch.Size([10])
###
***


对应的参数如下:
params = ***
torch.Size([6, 1, 3, 3])
torch.Size([6])
torch.Size([16, 6, 3, 3])
torch.Size([16])
torch.Size([120, 576])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])

In [3]:
input = torch.randn((1,1,32,32),requires_grad = True)
#前面的input中加上对应的属性requires_grad = True的时候，
#反向传播才能看到input的最终值
out = net(input)
print(out)
#前面为w=[10,84],乘上对应的矩阵[1,84],结果为10,注意前面乘上的
#w权重矩阵放在前面!!!

forward = ***
torch.Size([1, 6, 15, 15])
torch.Size([1, 16, 6, 6])
num_flat_features
size = ***
torch.Size([16, 6, 6])
num_features = ***
576
torch.Size([1, 576])
torch.Size([1, 120])
torch.Size([1, 84])
torch.Size([1, 10])
tensor([[-0.0914,  0.0550,  0.0276,  0.0287, -0.1046, -0.1136,  0.1201, -0.0392,
          0.0405, -0.0511]], grad_fn=<AddmmBackward>)


In [4]:
net.zero_grad()
#backward可能会积累梯度剩下，所以你需要提前进行
#zero_grad()对它进行操作
print('before backward')
print(out)
#将所有的梯度化为零并且反向传播使用随机的梯度
out.backward(torch.randn(1,10))
#到目前为止，只对浮点型数值使用backward函数
#默认的require_grad的值为False，如果一个require
#_grad的值为True，那么所有依附它的require_grad
#的对应值都为True
print('after backward')
print(input.grad)

before backward
tensor([[-0.0914,  0.0550,  0.0276,  0.0287, -0.1046, -0.1136,  0.1201, -0.0392,
          0.0405, -0.0511]], grad_fn=<AddmmBackward>)
after backward
tensor([[[[-0.0002, -0.0003,  0.0005,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0002, -0.0008, -0.0002,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0003,  0.0010, -0.0014,  ...,  0.0026,  0.0000,  0.0000],
          ...,
          [ 0.0003, -0.0003, -0.0011,  ...,  0.0010,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]]])


If any of tensors are non-scalar and require gradient,then the Jacobian-vector product would be computed.
如果tensors不为常量需要梯度，Jacobian-vector乘积需要进行计算
关于Jacobian-vector可以阅读相应的博文https://zhuanlan.zhihu.com/p/65609544
Tensor变量的requires_grad属性默认为False，若一个节点requires_grad被设置为True，那么所有依赖它的节点requires_grad都为True

In [7]:
output = net(input)
target = torch.randn(10)
target = target.view(1,-1)
print('target = ***')
print(target)
criterion = nn.MSELoss()
#MSELoss对应的为均方误差函数
loss = criterion(output,target)
print(loss)

forward = ***
torch.Size([1, 6, 15, 15])
torch.Size([1, 16, 6, 6])
num_flat_features
size = ***
torch.Size([16, 6, 6])
num_features = ***
576
torch.Size([1, 576])
torch.Size([1, 120])
torch.Size([1, 84])
torch.Size([1, 10])
target = ***
tensor([[-0.5896, -0.1096,  0.4998, -1.6665, -0.1585,  0.8958, -1.9492,  0.8551,
          1.4342, -0.7292]])
tensor(1.1878, grad_fn=<MseLossBackward>)
